In [1]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras import layers, models, callbacks


## Word2Vec

In [2]:
# === 1. 텍스트 전처리 ===
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"[^a-z\s]", "", text)  # 알파벳과 공백만
    tokens = word_tokenize(text)
    tokens = [t for t in tokens if t not in stopwords.words("english")]
    return tokens

In [3]:
# === 2. 데이터 로드 및 전처리 ===
df = pd.read_json("emb_cls.json")
df["tokens"] = df["review_text"].apply(preprocess_text)

In [4]:
df.head()

,review_text,fake,cleaned_text,bert_cls,roberta_cls,tokens
0,Awesome place to eat! Great atmosphere and fri...,1,Awesome place to eat Great atmosphere and frie...,"[0.0792026445, -0.0128625417, 0.3162086904, 0....","[-0.0814717263, 0.0890230536, -0.0438717902, -...","[awesome, place, eat, great, atmosphere, frien..."
1,"Great lunch date place! Food is amazing, not a...",1,Great lunch date place Food is amazing not a l...,"[-0.0047885552, 0.108762987, 0.0148257306, -0....","[-0.05344918, 0.0875567347, -0.0044471626, -0....","[great, lunch, date, place, food, amazing, lon..."
2,Best takeout Thai in the area and at a good pr...,1,Best takeout Thai in the area and at a good pr...,"[-0.1299686283, 0.1433178782, -0.0055814832, -...","[-0.030115934100000002, 0.0927834213, -0.01960...","[best, takeout, thai, area, good, price, highl..."
3,we been here few times for birthdays and coupl...,1,we been here few times for birthdays and coupl...,"[-0.0104819108, -0.207980454, 0.4680296183, -0...","[-0.0079873865, 0.019187642300000002, -0.02611...","[times, birthdays, couple, anniversarys, reaso..."
4,Excellent food and service. Yummy Delicious!!!...,1,Excellent food and service Yummy Delicious Flo...,"[-0.3778436184, -0.2384559363, -0.0925718844, ...","[-0.0456091613, 0.0996065363, -0.0168792922, -...","[excellent, food, service, yummy, delicious, f..."


In [5]:
# === 3. Word2Vec 학습 ===
w2v_model = Word2Vec(sentences=df["tokens"], vector_size=100, window=5, min_count=2, workers=4)
word_vectors = w2v_model.wv
vocab = word_vectors.key_to_index

In [6]:
# === 4. 토크나이저 및 시퀀스 ===
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df["tokens"])
sequences = tokenizer.texts_to_sequences(df["tokens"])
word_index = tokenizer.word_index

max_len = 100
X_seq = pad_sequences(sequences, maxlen=max_len)
y = df['fake'].values

In [7]:
# === 5. 임베딩 매트릭스 구성 ===
embedding_dim = 100
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))

for word, i in word_index.items():
    if word in word_vectors:
        embedding_matrix[i] = word_vectors[word]

In [8]:
print(embedding_matrix.shape)

(81611, 100)


In [9]:

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras import layers, models, callbacks

from tensorflow.keras.models import Model
from keras.layers import Input, Embedding, SimpleRNN, Dense, Dropout

from tensorflow.keras.optimizers import Adam # 학습률 조절
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.callbacks import EarlyStopping # 검증 손실 개선되지 않으면 학습 자동 멈춤, 과적함 방지.
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error # 결과 평가
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [10]:
# === 6. RNN 모델 ===
def build_rnn_model():
    input_layer = layers.Input(shape=(max_len,))
    embedding_layer = layers.Embedding(input_dim=len(word_index)+1,
                                       output_dim=embedding_dim,
                                       weights=[embedding_matrix],
                                       input_length=max_len,
                                       trainable=False)(input_layer)

    x = SimpleRNN(128, return_sequences=False, name='Basic_RNN_1')(embedding_layer)
    output = layers.Dense(1, activation='sigmoid')(x)

    model = models.Model(inputs=input_layer, outputs=output)
    model.summary()
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

### 모델 학습

In [11]:
def train_model(X, y):
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.2, stratify=y, random_state=42)

    model = build_rnn_model()
    early_stop = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    model.fit(X_train, y_train,
              validation_data=(X_val, y_val),
              epochs=1,
              batch_size=128,
              callbacks=[early_stop])

    y_pred_prob = model.predict(X_val)
    y_pred = (y_pred_prob >= 0.5).astype(int)

    print(f"Accuracy : {accuracy_score(y_val, y_pred):.4f}")
    print(f"Precision: {precision_score(y_val, y_pred):.4f}")
    print(f"Recall   : {recall_score(y_val, y_pred):.4f}")
    print(f"F1 Score : {f1_score(y_val, y_pred):.4f}")

    return model

In [12]:
model = train_model(X_seq, y)

/opt/anaconda3/envs/nlp_env/lib/python3.9/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 100, 100)       │     8,161,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Basic_RNN_1 (SimpleRNN)         │ (None, 128)            │        29,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,190,541 (31.24 MB)

 Trainable params: 29,441 (115.00 KB)

 Non-trainable params: 8,161,100 (31.13 MB)

500/500 ━━━━━━━━━━━━━━━━━━━━ 14s 27ms/step - accuracy: 0.6073 - loss: 0.6719 - val_accuracy: 0.6929 - val_loss: 0.5911
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step
Accuracy : 0.6929
Precision: 0.7348
Recall   : 0.6036
F1 Score : 0.6628


In [13]:
# from tensorflow.keras.utils import plot_model
# plot_model(model, show_shapes=True)